In [1]:
from model import *
from AliLoader import *
from ALI_Out import *

from torch.utils.data import DataLoader
import torchvision.datasets as dset
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import manifold
from sklearn import metrics
from scipy import stats
from AliMisc import *

from skopt.space import Real, Integer
from skopt import Optimizer

import seaborn as sns

In [2]:
def GetAlphaRedMap(cmap):
    #Print Rebuild
    # Get the colormap colors
    AlphaRed = cmap(np.arange(cmap.N))
    # Set alpha
    AlphaRed[:,-1] = np.linspace(0, 1, cmap.N)
    # Create new colormap
    AlphaRed = ListedColormap(AlphaRed)
    return(AlphaRed)
AlphaRed = GetAlphaRedMap(plt.cm.Reds)
AlphaBllue = GetAlphaRedMap(plt.cm.Blues)

In [3]:

LS = 512 #Latent Space Size
ColorsNumber = 1 #Number of color (always 1 for x-ray)
isize = 64

name = "Exp_64_512_0.00001_RandomLabel_4.0"

datadir = "./ChestXray-NIHCC-2/"

ExpDir = "./model/"+name



Load dataset

In [4]:
batch_size = 25
TestDF = pd.read_csv(ExpDir+"/TestImagesInfo.csv")


PreProDir = datadir+"PreProcess/Size"+str(isize)
test_dataset = XrayDatasetTensor(PreProDir+"/Tensor"+str(isize)+".pt",PreProDir+"/AllImagesInfo.csv",list(TestDF["name"]))
RealLoader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

#DiseaseDF = TestDF[TestDF["no_finding"] != 1]
#disease_dataset = XrayDatasetTensor(PreProDir+"/Tensor"+str(isize)+".pt",PreProDir+"/AllImagesInfo.csv",list(DiseaseDF["name"]))
#DiseaseLoader = DataLoader(disease_dataset, shuffle=False, batch_size=batch_size)
#len(disease_dataset)

FileNotFoundError: [Errno 2] No such file or directory: './ChestXray-NIHCC-2/PreProcess/Size64/Tensor64.pt'

In [ ]:
OtherXRayDir = "./chest_xray//*/*/*.jpeg"
Pneunomia = PneunomiaXray(OtherXRayDir, isize=isize,nrows=100)
PneuLoader = DataLoader(Pneunomia, shuffle=False, batch_size=batch_size)

In [ ]:
OtherXRayDir = "./reconstruct/*.jpeg"
APneunomia = PneunomiaXray(OtherXRayDir, isize=isize,nrows=100)
APneuLoader = DataLoader(APneunomia, shuffle=False, batch_size=batch_size)

In [ ]:
print(len(Pneunomia),len(APneunomia))

In [ ]:

CP = -2 #Checkpoint to load (-2 for latest one, -1 for last epoch)
DisX,DisZ,DisXZ,GenZ,GenX,CP,DiscriminatorLoss,AllAUCs = GenModel(isize,LS,-2,ExpDir,name,ColorsNumber=ColorsNumber)

DisX = DisX.eval()
DisZ = DisZ.eval()
DisXZ = DisXZ.eval()
GenZ = GenZ.eval()
GenX = GenX.eval()



In [ ]:
def RecLoss(GenX,GenZ,X):
    GenX.eval()
    GenZ.eval()
    Z = GenZ(X)
    Xr = GenX(Z)
    
    DiffX = Xr - X
    DiffX = DiffX.detach().numpy()
    DiffX = np.power(DiffX,2)
    RecLoss = [np.sqrt(np.mean(x)) for x in DiffX]
    Xr = Xr.detach().numpy()
    Z = Z.detach().numpy()
    Z = Z.reshape(Z.shape[0],Z.shape[1])
    return(RecLoss,DiffX,Xr,Z)
#Generate Alpha Red Map (so transparent) for error


In [ ]:
import time

In [ ]:

#AllLoader = [RealLoader,MNIST_loader,otherxray,hflip,vflip,randray,DiseaseLoader]
#Names = ["ChestXray","MNIST","MURA","hFlip","vFlip","Permuted","Disease"]

AllLoader = [RealLoader,PneuLoader,APneuLoader]
Names = ["ChestXray","Pneumonia","aPneumonia"]


#AllLoader = [RealLoader]
#Names = ["ChestXray"]

BenchData = dict()
test_size = 100
for (load,name) in zip(AllLoader,Names):
    BenchData[name] = dict()
    
    print(name)
    
    AllRecL = []
    AllX = []
    AllXr = []
    AllPath = []
    AllErr = []
    AllLab = []
    AllZ = []
    endtime = time.time()
    for Xi,path in load:
        
        if name == "MNIST":
            path = path.detach().numpy()
        inittime = time.time()
        #print(inittime-endtime)
        Xn = Xi*2.0 - 1
        Rl,DiffX,Xr,Z = RecLoss(GenX,GenZ,Xn)
        AllErr += list(DiffX)
        AllXr += list(Xr)
        AllRecL += Rl
        AllX += list(Xn.detach().numpy())
        AllZ += list(Z)
        if len(path) == 2:
            AllPath += list([p.split("/")[-1] for p in path[0]])
            AllLab += list(path[1])
        else:
            AllLab += list(path)
        plt.imshow(DiffX[0][0],cmap="gray")
        plt.show()
        
        plt.imshow(Xr[0][0],cmap="gray")
        plt.show()
        break
        if len(AllRecL) > test_size:
            AllRecL = AllRecL[:test_size]
            AllErr = AllErr[:test_size]
            AllPath = AllPath[:test_size]
            AllLab = AllLab[:test_size]
            AllX = AllX[:test_size]
            AllZ = AllZ[:test_size]
            AllXr = AllXr[:test_size]
            break
        endtime = time.time()
    BenchData[name]["RL"]   = AllRecL
    BenchData[name]["Diff"] = AllErr
    BenchData[name]["Recon"] = AllXr
    BenchData[name]["Path"] = AllPath
    BenchData[name]["Label"] = AllLab
    
    BenchData[name]["OriImg"] = AllX
    BenchData[name]["Z"] = AllZ


In [ ]:
fig = plt.figure(figsize=(16,8))
c = 1
for i,name in enumerate(Names):
    TestRL = BenchData[name]["RL"]
    TestXr = BenchData[name]["Recon"]
    TestX = BenchData[name]["OriImg"]
    TestErr = BenchData[name]["Diff"]
    for i in range(3):
        plt.subplot(6,12,c)
        plt.imshow(TestX[i][0],cmap="gray")
        c += 1
        
        plt.subplot(6,12,c)
        plt.imshow(TestXr[i][0],cmap="gray",vmax=1)
        c += 1
        
        plt.subplot(6,12,c)
        plt.imshow(TestErr[i][0],cmap=AlphaRed)
        plt.title("%.2f" % (TestRL[i]))
        c += 1
        c += 1
        
plt.show()


In [ ]:

fig = plt.figure(figsize=(15.5,8))
SNScol = sns.color_palette()

for i,name in enumerate(Names):
    #Get Reconstruction loss
    TestRL = BenchData[name]["RL"]
    RealRL = BenchData["ChestXray"]["RL"]
    
    #Concat
    CatRl = RealRL + TestRL
    
    #Gen Label
    Lab = [1]*len(RealRL)+[0]*len(TestRL)
    
    #Get AUC
    fpr, tpr, thresholds = metrics.roc_curve(Lab,-np.array(CatRl), pos_label=1)
    auc = metrics.auc(fpr, tpr)
    lab =  "%s %.2f" % (name,auc)
    if name == "ChestXray":
        lab = name
    #plt.hist(TestRL,label=lab, density=True,histtype="step",bins=20,color=SNScol[i])
    sns.kdeplot(TestRL, shade=True,label=lab,color=SNScol[i])
#plt.legend(fontsize = 30,frameon=False,prop={"family":"Sans Serif"})
plt.legend(fontsize = 15,frameon=False)

plt.xlabel("Reconstruction Loss",size=20)
plt.ylabel("Density",size=20)
plt.xticks(size=20)
plt.show()

In [ ]:
pred = pd.read_csv("predictions.csv").merge( pd.read_csv("uncertainty.csv"),on="Image name")


In [ ]:
pred["True label Pneumonia"] = pred["True label Pneumonia_y"]

In [ ]:
fig = plt.figure(figsize=(8,8))
fpr, tpr, thresholds = metrics.roc_curve(pred["True label Pneumonia"],np.array(pred["Pneumonia"]), pos_label=1)
auc = metrics.auc(fpr, tpr)
sns.kdeplot(pred["Pneumonia"][pred["True label Pneumonia"] == 0], shade=True,label="Healthy")
sns.kdeplot(pred["Pneumonia"][pred["True label Pneumonia"] == 1], shade=True,label="Pneumonia")
plt.title("AUC = %.2f" % (auc))
plt.show()


In [ ]:
subdf = pd.DataFrame([BenchData["Pneumonia"]["RL"],BenchData["Pneumonia"]["Path"]]).transpose()

In [ ]:
subdf.columns = ["RL","Name"]
subdf.head()

In [ ]:
test = pred.transpose().to_dict()

In [ ]:
for k in test.keys():
    for (rl,p) in zip(BenchData["Pneumonia"]["RL"],BenchData["Pneumonia"]["Path"]):
        if p in test[k]['Image name']:
            test[k]["RL"] = rl
            break


In [ ]:
MergePred = pd.DataFrame(test).transpose().sort_values(by="RL")
MergePred = MergePred.apply(pd.to_numeric, errors='ignore')

In [ ]:
ToPrint = 1
x = []
y = []
for i in range(400):
    cut = 1.4 - i / 400
    subdf = MergePred[MergePred["RL"] < cut]
    if len(subdf) < 100:
        break
    fpr, tpr, thresholds = metrics.roc_curve(subdf["True label Pneumonia"],subdf["Pneumonia"], pos_label=1)
    auc = metrics.auc(fpr, tpr)
    x.append(len(subdf)/len(MergePred)*100)
    y.append(auc)
    if cut <= 0.6:
        if ToPrint == 1:
            plt.scatter(len(subdf)/len(MergePred)*100,auc)
            plt.text(len(subdf)/len(MergePred)*100,auc+0.0055,"%.2f" % (cut),horizontalalignment='center')
            ToPrint = 0
plt.plot(x,y,label="RecLoss outliers")


ToPrint = 1
x = []
y = []
for i in range(400):
    cut = 0.3 - i / 2000
    subdf = MergePred[MergePred["stdPneumonia"] < cut]
    #print(cut,len(subdf))
    if len(subdf) < 100:
        print(i,cut,len(subdf))
        break
    fpr, tpr, thresholds = metrics.roc_curve(subdf["True label Pneumonia"],subdf["Pneumonia"], pos_label=1)
    auc = metrics.auc(fpr, tpr)
    x.append(len(subdf)/len(MergePred)*100)
    y.append(auc)
    
plt.plot(x,y,label="MC drop")
plt.legend()
plt.xlabel("Percent Retain")
plt.ylabel("Pneumoia AUROC")
plt.show()

In [ ]:
MergePred["NormPneumonia"] = MergePred["Pneumonia"]/MergePred["Pneumonia"].max()
MergePred["Error"] = np.power(MergePred["NormPneumonia"]-MergePred["True label Pneumonia"],2)

In [ ]:
ax = sns.kdeplot(MergePred["Pneumonia"][MergePred["True label Pneumonia"] == 0], MergePred["RL"][MergePred["True label Pneumonia"] == 0],
           cmap=AlphaBllue, shade=True, shade_lowest=False
           )
ax = sns.kdeplot(MergePred["Pneumonia"][MergePred["True label Pneumonia"] == 1], MergePred["RL"][MergePred["True label Pneumonia"] == 1],
           cmap=AlphaRed, shade=True, shade_lowest=False
           )
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
fpr, tpr, thresholds = metrics.roc_curve(pred["True label Pneumonia"],np.array(pred["Pneumonia"]), pos_label=1)
auc = metrics.auc(fpr, tpr)
sns.kdeplot(pred["Pneumonia"][pred["True label Pneumonia"] == 0], shade=True,label="Healthy")
sns.kdeplot(pred["Pneumonia"][pred["True label Pneumonia"] == 1], shade=True,label="Pneumonia")
plt.title("AUC = %.2f" % (auc))


subdf = MergePred[MergePred["RL"] < 0.6]
fpr, tpr, thresholds = metrics.roc_curve(subdf["True label Pneumonia"],np.array(subdf["Pneumonia"]), pos_label=1)
auc = metrics.auc(fpr, tpr)
sns.kdeplot(subdf["Pneumonia"][subdf["True label Pneumonia"] == 0], shade=True,label="Healthy Out")
sns.kdeplot(subdf["Pneumonia"][subdf["True label Pneumonia"] == 1], shade=True,label="Pneumonia Out")
plt.title("AUC = %.2f" % (auc))
plt.xlabel("RecLoss")
plt.show()


In [ ]:
fig = plt.figure(figsize=(8,8))
fpr, tpr, thresholds = metrics.roc_curve(MergePred["True label Pneumonia"],np.array(MergePred["RL"]), pos_label=1)
auc = metrics.auc(fpr, tpr)
sns.kdeplot(MergePred["RL"][MergePred["True label Pneumonia"] == 0], shade=True,label="Healthy")
sns.kdeplot(MergePred["RL"][MergePred["True label Pneumonia"] == 1], shade=True,label="Pneumonia")
plt.title("AUC = %.2f" % (auc))
plt.xlabel("RecLoss")
plt.show()


In [ ]:

plt.scatter(MergePred["stdPneumonia"],MergePred["RL"],s=0.2,
            c=MergePred["Error"],cmap="cool")
plt.colorbar()
plt.xlabel("MC Dropout std")
plt.ylabel("Reconstruction Loss")

plt.show()

plt.scatter(MergePred["stdPneumonia"],MergePred["Error"],s=0.2)
plt.xlabel("MC Dropout std")
plt.ylabel("Error")
plt.show()

plt.scatter(MergePred["RL"],MergePred["Error"],s=0.2)
plt.xlabel("Rec Loss")
plt.ylabel("Error")
plt.show()


In [ ]:
plt.scatter(MergePred["stdPneumonia"],MergePred["Pneumonia"])
plt.xlabel("stdPneumonia")
plt.ylabel("Pneumonia")

In [ ]:
test = pd.read_csv("alignedprediction.csv")
fig = plt.figure(figsize=(8,8))
fpr, tpr, thresholds = metrics.roc_curve(test["True label Pneumonia"],np.array(test["Pneumonia"]), pos_label=1)
auc = metrics.auc(fpr, tpr)
sns.kdeplot(test["Pneumonia"][test["True label Pneumonia"] == 0], shade=True,label="Healthy")
sns.kdeplot(test["Pneumonia"][test["True label Pneumonia"] == 1], shade=True,label="Pneumonia")
plt.title("AUC = %.2f" % (auc))
plt.show()

In [ ]:
fig = plt.figure(figsize=(15.5,9))
c = 1
for i,name in enumerate(["aPneumonia"]):
    RL = BenchData[name]["RL"]
    Diff = BenchData[name]["Diff"] 
    img = BenchData[name]["OriImg"]
    SortInd = np.argsort(RL)[::-1]
    RankPrint = range(25)
    for rp in RankPrint:
        NowInd = SortInd[rp]
        plt.subplot(5,10,c)
        plt.title("%.2f" % (RL[NowInd]))
        plt.imshow(img[NowInd][0],cmap="gray")
        #plt.imshow(Diff[NowInd][0],cmap=AlphaRed,vmin=0, vmax=2)
        plt.axis("off")
        
        c += 1
        
    RankPrint = (np.array([len(SortInd)]*25) - np.array(range(25))-1)[::-1]
    for rp in RankPrint:
        NowInd = SortInd[rp]
        plt.subplot(5,10,c)
        plt.title("%.2f" % (RL[NowInd]))
        plt.imshow(img[NowInd][0],cmap="gray")
        #plt.imshow(Diff[NowInd][0],cmap=AlphaRed,vmin=0, vmax=2)
        plt.axis("off")
        
        c += 1
plt.show()

In [ ]:
len(SortInd)

In [ ]:
def OutScore(DisX,DisZ,DisXZ,GenZ,GenX,X):
    
    
    z = GenZ(X)
    Xr = GenX(z)
    
    #CatT= torch.cat((DisZ(z), DisX(X)), 1)
    #PredFalse = DisXZ(CatT)
    
    DiffX = Xr - X
    DiffX = DiffX.detach().numpy()
    DiffX = np.power(DiffX,2)
    RecLoss = [np.sqrt(np.mean(x)) for x in DiffX]
    #PredFalse = PredFalse.detach().numpy()[0][0][0]
    #return(RecLoss,PredFalse)
    return(RecLoss)

In [ ]:
def TransformPImg(pim,inputsize,ar):
    tx = ar[0]
    ty = ar[1]
    rot = ar[2]
    sc = ar[3]
    brightness_factor = ar[4]
    contrast_factor = ar[5]
    tim = transforms.functional.affine(pim,angle=rot, translate=[tx,ty], 
                                                   scale=1, shear=0, resample=0, fillcolor=0)
    
    tim = transforms.functional.adjust_brightness(tim, brightness_factor)
    tim = transforms.functional.adjust_contrast(tim, contrast_factor)
    data_transforms = transforms.Compose([transforms.CenterCrop(sc)])
    ftim = data_transforms(tim)
    reim = transforms.functional.resize(ftim,inputsize)
    tim = transforms.functional.to_tensor(reim)
    tim = tim.reshape(1,1,inputsize,inputsize)
    return(ftim,tim)
    



In [ ]:



imgs = BenchData["Pneumonia"]["Path"]
RL = BenchData["Pneumonia"]["RL"]
RandInt = 20
Explore = 10


RandInt = 5
Explore = 10


#RandInt = 10
#Explore = 2

fig = plt.figure(figsize=(15.5,8))
c = 0


SortInd = np.argsort(RL)[::-1]
RankPrint = [0,1,2,3,4,5,6,7,int(len(RL)/4),int(len(RL)/2),int(len(RL)/4*3),len(RL)-5,len(RL)-4,len(RL)-3,len(RL)-2,len(RL)-1]
for rp in RankPrint:
    ind = SortInd[rp]
    imgf = BenchData["Pneumonia"]["Path"][ind]
    ptf = glob.glob("./chest_xray/*/*/*"+ imgf)[0]
    im = misc.imread(ptf)
    if len(im.shape) > 2:
        im = im[:, :, 0]
    #Add color chanel
    im = im[:,:,None]

    padding = 0
    if im.shape[0] > im.shape[1]:
        padding = (int((im.shape[0]-im.shape[1])/2),0)
    else:
        padding = (0,int((im.shape[1]-im.shape[0])/2))

    data_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Pad(padding,fill=0)
    ])
    pim = data_transforms(im)
    
    #pim = transforms.functional.to_pil_image(im)
    
    space  = [Integer(-200, 200, name='tx'),
          Integer(-200, 200, name='ty'),
          Real(-5,5, name='rot'),
          Integer(int(np.max(im.shape)/2), np.max(im.shape), name='sc'),
          Real(0.8,1.2, name='Bright'),
          Real(0.8,1.2, name='Contrast')
          ]
   
    
    
    opt = Optimizer(space)

    #Get random value
    
    
    Fimg = []
    Errs = []
    for it in range(Explore):
        RandTest = []
        if it == 0:
            RandTest.append([0,0,0,np.max(im.shape),1,1])
            
        totest = opt.ask(n_points=RandInt)
        RandTest += totest
        TensorTsc = torch.tensor([])
        for i in range(RandInt):
            FullImg,TensorImg = TransformPImg(pim,isize,RandTest[i])
            Fimg.append(FullImg)
            TensorTsc = torch.cat((TensorTsc,TensorImg),0)
        rl = OutScore(DisX,DisZ,DisXZ,GenZ,GenX,TensorTsc*2.0-1.0)
        for i in range(RandInt):
            opt.tell(RandTest[i], rl[i])
            #print(rl[i],RandTest[i])
        Errs += rl
        
        tsort = np.argsort(Errs)
        Errs = [Errs[tsort[0]]]
        Fimg = [Fimg[tsort[0]]]
        BestXi = opt.Xi[np.argsort(opt.yi)[0]]
        print(it,BestXi,np.sort(opt.yi)[0],BenchData["Pneumonia"]["RL"][ind])
    c += 1
    plt.subplot(4,8,c)
    plt.imshow(pim,cmap="gray")
    plt.title("%.2f" % (BenchData["Pneumonia"]["RL"][ind]))
    plt.axis("off")  
    c += 1
    plt.subplot(4,8,c)    
    plt.imshow(Fimg[0],cmap="gray")
    plt.title("%.2f" % (Errs[0]))
    plt.axis("off")
    
    BestXi = opt.Xi[np.argsort(opt.yi)[0]]
    print(BestXi,np.sort(opt.yi)[0],BenchData["Pneumonia"]["RL"][ind])
plt.show()

In [ ]:
Fimg